# **Random Forest IT-1**

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# 1. Cargar la base ya preprocesada
df = pd.read_csv("df_preprocesado_final.csv")

print("Shape del dataframe:", df.shape)
print("\nPrimeras columnas:", df.columns[:15])
print("\nColumnas relacionadas con el índice:")
print([c for c in df.columns if "IARD" in c])


Shape del dataframe: (260, 164)

Primeras columnas: Index(['2_edad', '12_horas_pedagogicas_semanales',
       '11_años_experiencia_docente', '13_promedio_estudiantes_curso',
       '1º Medio', '2º Medio', '3º Medio', '4º Medio', '7º Básico',
       '8º Básico', '1_genero_Femenino', '1_genero_Masculino',
       '1_genero_Prefiero no decirlo',
       '5_tipo_establecimiento_Administración Delegada',
       '5_tipo_establecimiento_Administración delegada'],
      dtype='object')

Columnas relacionadas con el índice:
['IARD_global']


In [ ]:
# Celda 2: Definir y, construir X y dejar solo columnas numéricas

# 2. Definir variable objetivo (target)
target = "IARD_global"

# y = índice global (variable dependiente)
y = df[target]

# 3. Construir X excluyendo el índice global
cols_excluir = [target]

X = df.drop(columns=cols_excluir, errors="ignore")

print("Shape X antes de filtrar tipos:", X.shape)

# 4. Nos quedamos solo con columnas numéricas (incluye tus dummies 0/1)
X = X.select_dtypes(include=["number"]).copy()

print("Shape X solo numérica:", X.shape)
print("\nEjemplo de columnas numéricas:")
print(X.columns[:20])


Shape X antes de filtrar tipos: (260, 163)
Shape X solo numérica: (260, 163)

Ejemplo de columnas numéricas:
Index(['2_edad', '12_horas_pedagogicas_semanales',
       '11_años_experiencia_docente', '13_promedio_estudiantes_curso',
       '1º Medio', '2º Medio', '3º Medio', '4º Medio', '7º Básico',
       '8º Básico', '1_genero_Femenino', '1_genero_Masculino',
       '1_genero_Prefiero no decirlo',
       '5_tipo_establecimiento_Administración Delegada',
       '5_tipo_establecimiento_Administración delegada',
       '5_tipo_establecimiento_Corporación',
       '5_tipo_establecimiento_De administración delegada',
       '5_tipo_establecimiento_Municipal',
       '5_tipo_establecimiento_Particular Pagado',
       '5_tipo_establecimiento_Particular Subvencionado'],
      dtype='object')


In [ ]:
# Celda 3: limpiar NaN y separar train/test
from sklearn.model_selection import train_test_split
import numpy as np

# 5. Eliminar filas con NaN en X o en y
mask = X.notna().all(axis=1) & y.notna()
X_clean = X[mask]
y_clean = y[mask]

print("Filas usadas tras limpiar NaN:", X_clean.shape[0])

# 6. Train-test split (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(
    X_clean, y_clean,
    test_size=0.3,
    random_state=42
)

print("Tamaño train:", X_train.shape[0], " - Tamaño test:", X_test.shape[0])


Filas usadas tras limpiar NaN: 260
Tamaño train: 182  - Tamaño test: 78


In [ ]:
# Entrenar el Random Forest (Celda 4)
from sklearn.ensemble import RandomForestRegressor

# 7. Definir el modelo de Random Forest en modo regresión
rf = RandomForestRegressor(
    n_estimators=500,
    random_state=42,
    n_jobs=-1,
    max_depth=None
)

# 8. Entrenar el modelo
rf.fit(X_train, y_train)


RandomForestRegressor(n_estimators=500, n_jobs=-1, random_state=42)

In [ ]:
# Evaluar el modelo (Celda 5)
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# 9. Predicciones
y_pred_train = rf.predict(X_train)
y_pred_test = rf.predict(X_test)

def resumen_metricas(y_true, y_pred, nombre=""):
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    print(f"== {nombre} ==")
    print("R²   :", round(r2, 3))
    print("MAE  :", round(mae, 2))
    print("RMSE :", round(rmse, 2))
    print()

resumen_metricas(y_train, y_pred_train, "Train")
resumen_metricas(y_test, y_pred_test, "Test")


== Train ==
R²   : 0.851
MAE  : 3.84
RMSE : 4.74

== Test ==
R²   : -0.077
MAE  : 10.11
RMSE : 12.16



In [ ]:
# Importancia de variables (Celda 6)
import pandas as pd

importances = rf.feature_importances_
feature_names = X_clean.columns

fi_df = pd.DataFrame({
    "variable": feature_names,
    "importancia": importances
}).sort_values("importancia", ascending=False)

print("Top 15 variables más importantes:")
print(fi_df.head(15))


Top 15 variables más importantes:
                                              variable  importancia
0                                               2_edad     0.147822
1                       12_horas_pedagogicas_semanales     0.099149
4                                             1º Medio     0.042891
2                          11_años_experiencia_docente     0.034360
5                                             2º Medio     0.032193
66                      10_asignatura_principal_Música     0.031575
59   10_asignatura_principal_Lenguaje y Comunicació...     0.031293
150            14_carrera_origen_Profesora Diferencial     0.026904
19     5_tipo_establecimiento_Particular Subvencionado     0.024108
20                         5_tipo_establecimiento_SLEP     0.022308
10                                   1_genero_Femenino     0.021644
28                         9_modalidad_educacion_Ambos     0.016950
61                  10_asignatura_principal_Matemática     0.016767
7             